In [1]:
# # install necessary packages for imports 
# %pip install scikit-image
# %pip install scikit-learn
# %pip install matplotlib
# %pip install cartopy
# %pip install folium
# %pip install mapbox_vector_tile
# %pip install lxml
# %pip install pandas
# %pip install owslib
# %pip install geopandas
# %pip install rasterio
# %pip install fiona
# %pip install ipyleaflet
# %pip install cairosvg # If needed, more specific install instructions for cairosvg: https://cairosvg.org/documentation/ 

In [2]:
# import os
# from io import BytesIO
# from skimage import io
# import requests
# import json
# import matplotlib.pyplot as plt
# import matplotlib.ticker as mticker
# import cartopy.crs as ccrs
# import cartopy
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
# import folium
# import urllib.request
# import urllib.parse
# import mapbox_vector_tile
# import xml.etree.ElementTree as xmlet
# import lxml.etree as xmltree
# from PIL import Image as plimg
# from PIL import ImageDraw
# import numpy as np
# import pandas as pd
# from owslib.wms import WebMapService
# from IPython.display import Image, display
# import geopandas as gpd
# from shapely.geometry import box
# import urllib.request
# import rasterio
# from rasterio.mask import mask
# from rasterio.warp import calculate_default_transform, reproject, Resampling
# from rasterio.plot import show
# import fiona
# from datetime import datetime, timedelta
# %matplotlib inline


In [3]:
# Connect to GIBS WMS Service
# wms = WebMapService('https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?', version='1.1.1')

In [4]:
import pandas as pd
from pathlib import Path

# MODIS fire detections (last 7 days, USA + Hawaii)
file_path = Path.cwd() / "datasets" / "MODIS_C6_1_USA_contiguous_and_Hawaii_7d.csv"
fires = pd.read_csv(file_path)

# url = "https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_Global_24h.csv"
# fires = pd.read_csv(url)

In [5]:
fires

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight
0,17.75904,-92.91803,321.99,2.18,1.43,2025-10-27,338,T,100,6.1NRT,293.99,55.45,N
1,17.75978,-92.92379,315.56,2.18,1.43,2025-10-27,338,T,91,6.1NRT,293.94,37.65,N
2,17.89660,-92.48100,306.09,2.37,1.49,2025-10-27,338,T,67,6.1NRT,294.44,17.72,N
3,17.89438,-92.50278,305.52,2.36,1.48,2025-10-27,338,T,65,6.1NRT,293.96,16.77,N
4,17.84301,-93.12038,362.81,2.11,1.41,2025-10-27,338,T,100,6.1NRT,295.73,276.15,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,36.77638,-118.64378,317.90,1.05,1.02,2025-11-03,2233,A,54,6.1NRT,292.26,15.75,D
2383,36.74765,-118.85920,307.62,1.04,1.02,2025-11-03,2233,A,59,6.1NRT,287.19,9.35,D
2384,36.78689,-118.63350,362.54,1.05,1.02,2025-11-03,2233,A,100,6.1NRT,297.56,98.63,D
2385,36.78539,-118.64479,392.26,1.05,1.02,2025-11-03,2233,A,100,6.1NRT,308.32,222.84,D


In [6]:
fires['acq_date'] = pd.to_datetime(fires['acq_date'])
fires['acq_time'] = pd.to_datetime(fires['acq_time'], format='%H%M').dt.time
fires

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight
0,17.75904,-92.91803,321.99,2.18,1.43,2025-10-27,03:38:00,T,100,6.1NRT,293.99,55.45,N
1,17.75978,-92.92379,315.56,2.18,1.43,2025-10-27,03:38:00,T,91,6.1NRT,293.94,37.65,N
2,17.89660,-92.48100,306.09,2.37,1.49,2025-10-27,03:38:00,T,67,6.1NRT,294.44,17.72,N
3,17.89438,-92.50278,305.52,2.36,1.48,2025-10-27,03:38:00,T,65,6.1NRT,293.96,16.77,N
4,17.84301,-93.12038,362.81,2.11,1.41,2025-10-27,03:38:00,T,100,6.1NRT,295.73,276.15,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,36.77638,-118.64378,317.90,1.05,1.02,2025-11-03,22:33:00,A,54,6.1NRT,292.26,15.75,D
2383,36.74765,-118.85920,307.62,1.04,1.02,2025-11-03,22:33:00,A,59,6.1NRT,287.19,9.35,D
2384,36.78689,-118.63350,362.54,1.05,1.02,2025-11-03,22:33:00,A,100,6.1NRT,297.56,98.63,D
2385,36.78539,-118.64479,392.26,1.05,1.02,2025-11-03,22:33:00,A,100,6.1NRT,308.32,222.84,D


| Column                      | Description                                                                                                                                        |
| --------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- |
| **latitude**, **longitude** | Geographic coordinates (in decimal degrees) of the detected fire pixel.                                                                            |
| **brightness**              | Measured brightness temperature (Kelvin) in the mid-infrared band (~4 µm). Higher values generally mean more intense heat.                         |
| **scan**, **track**         | Pixel size (degrees) in the scan (east–west) and track (north–south) directions — indicates spatial resolution or size of the detection footprint. |
| **acq_date**                | The date when the satellite detected the fire.                                                                                                     |
| **acq_time**                | Time of acquisition (in UTC) — usually minutes past midnight, not a local time.                                                                    |
| **satellite**               | Which MODIS satellite made the observation: **A** = Aqua, **T** = Terra.                                                                           |
| **confidence**              | Confidence score (0–100) estimating the likelihood that the pixel represents a real fire (rather than noise or a false alarm).                     |
| **version**                 | Processing version (e.g., “6.1NRT” = Collection 6.1, Near Real-Time product).                                                                      |
| **bright_t31**              | Brightness temperature (Kelvin) from the thermal IR band (~11 µm), which measures surface temperature rather than fire intensity.                  |
| **frp**                     | **Fire Radiative Power** (MW) — an estimate of the energy output of the fire. Higher FRP = larger or more intense fire.                            |
| **daynight**                | Indicates whether the detection was made during the **day (D)** or **night (N)**.                                                                  |
